In [9]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn
%matplotlib inline

In [10]:
# reading file
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [11]:
len(text)

1115394

In [12]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [13]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [14]:
char_to_idx = {ch:i for i, ch in enumerate(chars)}
idx_to_char = {i:ch for ch, i in char_to_idx.items()}

In [17]:
def encode(x):
    return [char_to_idx[i] for i in x]

def decode(x):
    return ''.join([idx_to_char[i] for i in x])

In [18]:
data = torch.tensor(encode(text), dtype = torch.long)
data.shape

torch.Size([1115394])

In [19]:
n1, n2 = int(len(text) * 0.8), int(len(text) * 0.9)
train_data = data[:n1]
val_data = data[n1:n2]
test_data = data[n2:]

In [20]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print(xb)
print(yb)

tensor([[57, 43, 10,  0, 35, 46, 39, 58],
        [42, 47, 57, 41, 53, 52, 58, 43],
        [39, 42, 63,  1, 14, 53, 52, 39],
        [61, 43, 56, 43,  1, 63, 53, 59]])
tensor([[43, 10,  0, 35, 46, 39, 58,  5],
        [47, 57, 41, 53, 52, 58, 43, 52],
        [42, 63,  1, 14, 53, 52, 39,  6],
        [43, 56, 43,  1, 63, 53, 59, 56]])


In [21]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

m = BigramLanguageModel(vocab_size)

idx = torch.zeros((1, 1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens = 100)[0].tolist()))


3Ym:BUuJ,w;.ygNu'nLnVkne.zVeusJZhd;hYykSh!:yupdsJTkxldzLrA&&mUjFjtVp
Edy;bZJslxW?xxxx&&l!P'UOUKWK&?;


In [22]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [23]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.6168501377105713


In [24]:
print(decode(m.generate(idx, max_new_tokens = 300)[0].tolist()))


vGL$NinVOntZokKLory.y?nV uxYNouCx&Iz!RFMwQAY?OSi
lk3BA:PpdoinVpWyuaniloiB&Ed;q yIREXJ:Q:wH,
WHOUFMkYfbdixycoeGv,WTkvetHP :CBzCah;h-&z.yI$QPDcumk$BOzuBFMigKit;DldIEZ;heldVOGLT BW
RA.qgmifMulheGOYCSN:wJyJVOfGAUqLs'PbyuldTKLuxQ3d;veuMEM!bwXIzulk?hy fakQ:GcSsp wLIs uzoGjIhvuCOz!d.bU
yHOva:bUKwdsTTonVywi


In [26]:
# CONSIDER THE FOLLOWING EXAMPLE:
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [28]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [30]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim = 1, keepdim = True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

In [34]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim = True)
xbow2 = wei @ x #(B, T, T) @ (B, T, C)

In [36]:
#  can also use softmax for batch maultiplication
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x